# Week 11: ROW_NUMBER

## What does ROW_NUMBER do?
The `ROW_NUMBER` function is a window function, i.e. a function that sorts the data into so-called "windows" or "partitions". You define the sorting and the partitions by using the keywords `ORDER BY` and `PARTITION BY` after the function call. The `ROW_NUMBER` function then numbers all rows in the result set, starting at 1 in ascending order. Each partition then starts at 1 again. 

## How can you work with ROW_NUMBER?
You can use the `ROW_NUMBER` function in the column list of your query if you want to see the assigned numbers:

In [0]:
SELECT
     [CountryID]
    ,ROW_NUMBER() OVER (ORDER BY [CountryID]) as [RowNumber]
    ,[CountryName]
    ,[FormalName]
    ,[IsoAlpha3Code]
FROM [WideWorldImporters].[Application].[Countries]

You will now see a column with the heading `RowNumber` and a consecutive number in the content. Great. Well. Let's see if we can do anything with it. If you look at the result, you can see that already early in the row of Albania the `CountryID` differs from the `RowNumber`. Now you can of course use the `ROW_NUMBER` function to find the records (well, the record) where both values match:

In [0]:
; WITH cte_rn as (
    SELECT
         [CountryID]
        ,ROW_NUMBER() OVER (ORDER BY [CountryID]) as [RowNumber]
        ,[CountryName]
        ,[FormalName]
        ,[IsoAlpha3Code]
    FROM [WideWorldImporters].[Application].[Countries]
)
SELECT 
     [CountryId]
    ,[CountryName]
    ,[FormalName]
    ,[IsoAlpha3Code]
FROM cte_rn 
WHERE [CountryID] = [RowNumber]

You can see that at the end there is only one line left, the one for Afghanistan. For all other lines, the number of lines differs from the ID. If you would want consecutive IDs, you could now update the table with the CTE and set the `CountryID` accordingly (provided that no `IDENTITY` column has been defined here, otherwise you have to work with `IDENTITY INSERT`).

This is all quite nice, but what can you really do with the `ROW_NUMBER` function? Probably the most common use of the `ROW_NUMBER` function in combination with CTE's is to filter duplicate rows from a result. One (admittedly quite stupid) query to see this is the following, from which we now want to filter the duplicates for the countries: 

In [0]:
SELECT 
     c.[CountryId]
    ,c.[CountryName]
    ,c.[FormalName]
    ,s.[StateProvinceID]
FROM [Application].[Countries] as c
LEFT JOIN [Application].[StateProvinces] as s 
ON c.[CountryID] = s.[CountryID]
ORDER BY c.CountryID

In the result set, you can see here that the USA appears several times (because several states are defined in the StateProvinces table). So now let's add a `ROW_NUMBER` that numbers the states within a country consecutively: 

In [0]:
SELECT 
     c.[CountryId]
    ,c.[CountryName]
    ,c.[FormalName]
    ,s.[StateProvinceID]
    ,ROW_NUMBER() OVER (PARTITION BY c.[CountryID] ORDER BY s.[StateProvinceID]) as [RowNumber]
FROM [Application].[Countries] as c
LEFT JOIN [Application].[StateProvinces] as s 
ON c.[CountryID] = s.[CountryID]

You can see in the result set that for all countries that do not know any states, only a `NULL` value appears as a state (so the `LEFT JOIN` had no hit), but for the USA 53 lines were entered (what a coincidence). Note that we have specified a `PARTITION BY` (unlike in the previous examples). This means that if the `CountryID` changes in the result set, our 'ROW_NUMBER' function will start counting from 1 again. 

If you now want to filter a row from this, you can take advantage of this behavior by keeping only the entries with a `ROW_NUMBER` of 1: 

In [0]:
; with cte_rn as ( 
    SELECT 
         c.[CountryId]
        ,c.[CountryName]
        ,c.[FormalName]
        ,s.[StateProvinceID]
        ,ROW_NUMBER() OVER (PARTITION BY c.[CountryID] ORDER BY s.[StateProvinceID]) as [RowNumber]
    FROM [Application].[Countries] as c
    LEFT JOIN [Application].[StateProvinces] as s 
    ON c.[CountryID] = s.[CountryID]
)
select 
     [CountryID]
    ,[CountryName]
    ,[FormalName]
    ,[StateProvinceID]
    ,[RowNumber]
FROM cte_rn
WHERE [RowNumber] = 1

Your found set now again contains only one row for the USA. 
With the help of this pattern you can now filter duplicates from tables in all possible and impossible situations. Note, however, that it would be better to write your queries in such a way that no unwanted duplicates occur. 
### References
- [Official Documentation from Microsoft](https://docs.microsoft.com/de-de/sql/t-sql/functions/row-number-transact-sql?view=sql-server-2017)
